# Introduction

The *peep* package contains some useful functions for interacting with the Python API of Google Earth Engine to download images, and also code to process these images and prepare them for analysis.

In particular, we build composites of the avalaible images during a time period to a single image, that we then divide into 32x32 pixel sub-images, which are saved as .npy files that can be used for analysis.

Before we can use GEE, we need to authenticate (assuming we have an account). We recomend use `gcloud` and authenticate before running this notebook.   

Another method for authentification is running

```
ee.Initialize()
```
and following instructions, however there seems to be a move from GEE to deprecate this method of authentication.

## Pipelines, Sequences, and Modules

In `peep`, we have the concept of a "Pipeline" for downloading and processing data from GEE.

A Pipeline is composed of one or more Sequences, which are in turn composed of Modules.

A Module is an class designed for one specific task (e.g. "download data from GEE", or "process tiff file and convert them into RGB images"), and they are generally grouped into Sequences such that one Module will work on the output of the previous one.  
So our standard Pipeline has:
* A Download Sequence consisting of ImageDownloader, ImageProcessor.   
* A weather Sequence consisting of WeatherDownloader, WeatherImageToJSON

For this project we are not dowloading weather, therefore we only will working with the ImageDownloader, and ImageProcessor.

### Running the full pipeline from the command-line

We have a couple of "entrypoints" (i.e. command-line commands) linked to functions in some `peep` scripts to help do this.  
* To configure and run a downloading-and-processing pipeline we run the command `peep_run_pipeline --config_file <some-config-file>`
* To generate the config file in the above command we have the command `peep_generate_config`.

Both these can accept multiple command-line arguments, and these can be seen with the `--help` argument:

In [ ]:
!peep_generate_config --help

For `pyveg_generate_config` any parameters it needs that are not provided as command-line arguments will be requested from the user, and the various allowed options will be provided, along with (in most cases) default values that will be used if the user just presses "enter".
However, although just running `pyveg_generate_config` with no arguments and then responding to the prompts is probably the easiest way to run it on the command line, this doesn't seem to work so well with Jupyter, so let's just provide all the arguments it needs:

In [ ]:
!peep_generate_config --configs_dir ../../peep/configs --collection_name Sentinel2 --left 532480 --bottom 174080 --right 542720 --top 184320 --start_date 2016-06-01 --end_date 2016-09-01 --time_per_point 3m --output_dir .


We can see from the output that a new config file has been written, and the command we should use to run with it.

In [ ]:
!peep_run_pipeline --config_file ../../peep/configs/config_Sentinel2_532480_0174080_542720_0184320_2016-06-01_2016-09-01_3m.py




So we just:
* Downloaded some Sentinel2 images from GEE: particularly bands B2, B3 and B4, as well as a COUNT band that shows how many images were used in the composite at pixel level. 
* Converted these raw tif images into RGB png (B2, B3 and B4) and a COUNT heatmap png.
* Split the RGB png and the COUNT band into 50x50 sub-images.
* Save RGB sub-images as arrays in .npy files
* Summarise statistics of COUNT sub-images (mean, min, max, etc) in json files.

There will now be a directory called "Sentinel2-532480-0174080-542720-0184320__[date_stamp]" in your current directory, with a subdirectory called "gee_532480_0174080_542720_0184320_Sentinel2". Within this there are directories "[date-start_date-end]/PROCESSED" that contain the images and "[date-start_date-end]/SPLIT" the sub-images files and "[date-start_date-end]/RAW" the original dowloaded bands as tiff files.

# Generating many configuration files  

If you want to generate a large number of configuration files from a dataset of geometries you can do this using the command  `peep_generate_config` and
a [geoparquet](https://pypi.org/project/geoparquet/) file (`bounds_file`).

The geoparquet file must contain a column named `geometry`, and each row correspond to an individual geometry (an example of this
file can be found as `images_1024.parquet` on the `testdata` directory which covers all the UK territory).  

`peep` is optimise to work with the British National Grid projection coordinates, therefore the file must be projected in this way.

In [ ]:
import geopandas 

bounds_gdf = geopandas.read_parquet('../../peep/testdata/images_1024.parquet')
bounds_gdf.to_crs("EPSG:27700")

If provided
the `--bounds_file` flag, the `peep_generate_config` script will read the geoparquet file, loop over its rows creating a config file
for each geometry.

If a column `on_land` is available in the geoparquet file, the script will filter only rows where the column is True.
If not this column is not found, the script will loop over all rows of the data.

Flags such as `start_date`, `end_date`, `time_per_point` must be provided as well, and it will be used for all the config files created.

Flags such as `configs_dir`, `output_dir` are optional, defining to write the output of the config files or downloads to a specified directory
with the default being the current directory.

To generate config files from the parquet file above:

In [2]:
!peep_generate_config --bounds_file ../../peep/testdata/images_1024.parquet --start_date 2018-04-01 --end_date 2018-10-01 --time_per_point 6m --configs_dir downloads_configs --output_dir downloads_output


                    output_location downloads_output/87-Sentinel2-000000-0890880-010240-0901120
                    collection: Sentinel2
                    left: 000000
                    bottom: 0890880
                    right: 010240
                    top: 0901120
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_87_Sentinel2_000000_0890880_010240_0901120_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_87_Sentinel2_000000_0890880_010240_0901120_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/88-Sentinel2-000000-0901120-010240-0911360
                    collection: Sentinel2
                   


                    output_location downloads_output/1714-Sentinel2-133120-0645120-143360-0655360
                    collection: Sentinel2
                    left: 133120
                    bottom: 0645120
                    right: 143360
                    top: 0655360
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_1714_Sentinel2_133120_0645120_143360_0655360_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_1714_Sentinel2_133120_0645120_143360_0655360_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/1715-Sentinel2-133120-0655360-143360-0665600
                    collection: Sentinel2
           

Wrote file 
  downloads_configs/config_2112_Sentinel2_163840_0819200_174080_0829440_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_2112_Sentinel2_163840_0819200_174080_0829440_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/2113-Sentinel2-163840-0829440-174080-0839680
                    collection: Sentinel2
                    left: 163840
                    bottom: 0829440
                    right: 174080
                    top: 0839680
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_2113_Sentinel2_163840_0829440_174080_0839680_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit thi


                    output_location downloads_output/2497-Sentinel2-194560-0860160-204800-0870400
                    collection: Sentinel2
                    left: 194560
                    bottom: 0860160
                    right: 204800
                    top: 0870400
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_2497_Sentinel2_194560_0860160_204800_0870400_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_2497_Sentinel2_194560_0860160_204800_0870400_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/2498-Sentinel2-194560-0870400-204800-0880640
                    collection: Sentinel2
           

Wrote file 
  downloads_configs/config_2803_Sentinel2_225280_0092160_235520_0102400_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_2803_Sentinel2_225280_0092160_235520_0102400_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/2804-Sentinel2-225280-0102400-235520-0112640
                    collection: Sentinel2
                    left: 225280
                    bottom: 0102400
                    right: 235520
                    top: 0112640
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_2804_Sentinel2_225280_0102400_235520_0112640_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit thi

...


                    output_location downloads_output/5597-Sentinel2-450560-0092160-460800-0102400
                    collection: Sentinel2
                    left: 450560
                    bottom: 0092160
                    right: 460800
                    top: 0102400
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_5597_Sentinel2_450560_0092160_460800_0102400_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_5597_Sentinel2_450560_0092160_460800_0102400_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/5598-Sentinel2-450560-0102400-460800-0112640
                    collection: Sentinel2
           


                    output_location downloads_output/5879-Sentinel2-471040-0378880-481280-0389120
                    collection: Sentinel2
                    left: 471040
                    bottom: 0378880
                    right: 481280
                    top: 0389120
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_5879_Sentinel2_471040_0378880_481280_0389120_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_5879_Sentinel2_471040_0378880_481280_0389120_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/5880-Sentinel2-471040-0389120-481280-0399360
                    collection: Sentinel2
           

Wrote file 
  downloads_configs/config_6267_Sentinel2_501760_0450560_512000_0460800_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_6267_Sentinel2_501760_0450560_512000_0460800_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/6268-Sentinel2-501760-0460800-512000-0471040
                    collection: Sentinel2
                    left: 501760
                    bottom: 0460800
                    right: 512000
                    top: 0471040
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_6268_Sentinel2_501760_0460800_512000_0471040_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit thi


                    output_location downloads_output/6756-Sentinel2-542720-0256000-552960-0266240
                    collection: Sentinel2
                    left: 542720
                    bottom: 0256000
                    right: 552960
                    top: 0266240
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_6756_Sentinel2_542720_0256000_552960_0266240_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_6756_Sentinel2_542720_0256000_552960_0266240_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/6757-Sentinel2-542720-0266240-552960-0276480
                    collection: Sentinel2
           


                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_7387_Sentinel2_593920_0215040_604160_0225280_2018-04-01_2018-10-01_6m.py
                    

                    output_location downloads_output/7388-Sentinel2-593920-0225280-604160-0235520
                    collection: Sentinel2
                    left: 593920
                    bottom: 0225280
                    right: 604160
                    top: 0235520
                    start_date: 2018-04-01
                    end_date: 2018-10-01
                    time_per_point: 6m
                      
Wrote file 
  downloads_configs/config_7388_Sentinel2_593920_0225280_604160_0235520_2018-04-01_2018-10-01_6m.py
We recommend that you add and commit this to your version control repository.

                    To run peep using this configuration, do:

                    peep_run_pipeline --config_file downloads_configs/config_7388_Sentin

There will now be a directory called "downloads_configs" in your current directory with 2745 config file correspoinding to each of the images in the UK that are over land. Each config file is for the 6 month period between April and October 2018, if you want to download another time period you need to run the command line above with different `start_date` and `end_date`. 

We can now use these config files to download data consecutively as it will be shown in the next section.

# Downloading data consecutively from multiple configuration files

You might want to download images from several configuration files in one go. For this you have to have all the config files from the images you want to download in the same directory.

The following script runs a loop and executes the `peep_run_pipeline` command on each available file found in the path where we created our config files in the section above.


In [4]:
!peep_run_pipeline_loop --config_dir downloads_configs

2022-09-13 11:05:45,555 [INFO] Running gee download using the command: peep_run_pipeline --config_file downloads_configs/config_3366_Sentinel2_266240_0655360_276480_0665600_2018-04-01_2018-10-01_6m.py
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting collection_name to COPERNICUS/S2
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting data_type to vegetation
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting RGB_bands to ['B4', 'B3', 'B2']
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting NIR_band to B8
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting mask_cloud to True
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting cloudy_pix_frac to 50
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting cloudy_pix_flag to CLOUDY_PIXEL_PERCENTAGE
2022-09-13 11:05:56,265 [INFO] Sentinel2: setting min_date to 2016-01-01
2022-09-13 11:05:56,266 [INFO] Sentinel2: setting max_date to 2022-01-01
2022-09-13 11:05:56,266 [INFO] Sentinel2: setting time_per_point to 1m
/Users/crangelsmith/opt/anaconda3/envs/geepi

It can take up to 60 hours to run all of the config files covering the UK at one time point. 